In [1]:
import tensorflow as tf
from tensorflow import keras

Lambda Functions

In [2]:
x=lambda a: a+10 # lambda input_parameters: action_return-results
x(4)

14

### Sentiment Analysis

In [3]:
tf.random.set_seed(42)

In [4]:
(X_train, y_test), (X_valid, y_test) = keras.datasets.imdb.load_data()

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
C:\Anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
C:\Anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant

In [5]:
X_train[0][:10]

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

#### Shakespeare’s work

In [6]:
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [7]:
print(shakespeare_text[:148])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



In [8]:
"".join(sorted(set(shakespeare_text.lower()))) # 13 + 26 = 39 different characters

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

In [9]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

In [11]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)

In [12]:
train_size = info.splits["train"].num_examples
test_size = info.splits["test"].num_examples

In [13]:
train_size, test_size

(25000, 25000)

In [14]:
for X_batch, y_batch in datasets["train"].batch(2).take(1):
    for review, label in zip(X_batch.numpy(), y_batch.numpy()):
        print("Review:", review.decode("utf-8")[:200], "...")
        print("Label:", label, "= Positive" if label else "= Negative")
        print()

Review: I gave 1 to this film. I can't understand how Ettore Scola,one of the greater directors of Italian cinema, made a film like this, so stupid and ridiculous! All the stories of the people involved in th ...
Label: 0 = Negative

Review: Several young Iranian women dress as boys and try to get into a World Cup qualifying match between Iran and Bahrain. When they're caught, they're penned in an area where the match remains within earsh ...
Label: 1 = Positive



In [15]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch